# Utilities

> A set of functions that I commonly use in notebooks

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import nptyping
import numpy as np

In [ ]:
#| export
def thread_macro(current_value, *funcs, identifier="self"):
    """Pipes current_value through each function in funcs.

    Each element in funcs is either a function or a list/tuple containing
    a function followed by its other arguments.
    This function imitates the Clojure as-if threading macro.

    Notes: By default current_value is threaded as the first argument of the
    function call. Yet, one can use the syntax [func, arg1, "self", arg2] (or
    (func, arg1, "self", arg2)) so that current_value will instead be threaded
    in whatever place "self" would be. If you need to, you can set this "self"
    identifier to a different value.
    """

    for func in funcs:
        if isinstance(func, (list, tuple)):
            place = 0
            for i, el in enumerate(func[1:]):
                if el == identifier:
                    place = i
                    func = [el for el in func if el != identifier]
            func, args1, args2 = func[0], func[1:place + 1], func[place + 1:]
            current_value = func(*args1, current_value, *args2)
        else:
            current_value = func(current_value)
    return current_value

In [ ]:
#| export
def broadcast_concatenate_axes(ax1, ax2):
    """Broadcast both numpy axes and concatenate along last dimension"""
    ax1new = ax1
    for _ in range(np.ndim(ax2) - 1):
        ax1new = ax1new[..., None, :]
    ax2new = ax2
    for _ in range(np.ndim(ax1) - 1):
        ax2new = ax2new[None, ..., :]
    ax1new = np.broadcast_to(ax1new,
                             (*ax1.shape[:-1], *ax2.shape[:-1], ax1.shape[-1]))
    ax2new = np.broadcast_to(ax2new,
                             (*ax1.shape[:-1], *ax2.shape[:-1], ax2.shape[-1]))
    ax = np.concatenate((ax1new, ax2new), axis=-1)
    return ax

In [ ]:
#| export
def build_parameter_grid_from_axes(axes):
    """Build a numpy array with all combinations of elements specified in axes.

    Each axis in axes gives an array of values that should be repeated for each
    value in the other axes. Primitive types and lists of primitive types are
    first promoted to numpy arrays.
    """

    dtypes = (float, int, bool, str)
    for i, axis in enumerate(axes):
        condition = isinstance(axis, dtypes) or all(
            isinstance(el, dtypes) for el in list(axis))
        axes[i] = np.array([axis]).T if condition else axis
    tensor = functools.reduce(broadcast_concatenate_axes, axes)
    return tensor.reshape((-1, tensor.shape[-1]))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()